In [ ]:
%%pyspark
df = spark.read.load('abfss://sampledataset@REPLACETHIS.dfs.core.windows.net/flight/weather-ref/AirportWeatherReferenceData.csv', format='csv', header=True, )
display(df.limit(10))

In [ ]:
df = df.na.fill('')

In [ ]:
display(df.limit(10))

In [ ]:
df.write.mode("overwrite").saveAsTable("default.weather")

In [ ]:
df

In [ ]:
%%sql
select distinct * from
(
select distinct 
month,
day,
latitude,
longitude,
ceil(time / 100) as hour,
cast((case wind_speed when null then 0.0 when 'M' then 0.0 else wind_speed end) as decimal(5,3)) as wind_speed,
cast((case sea_level_pressure when null then 0.0 when 'M' then 29.92 else sea_level_pressure end) as decimal(5,3)) as sea_level_pressure,
cast((case hourly_precip when '' then 0.0 when 'T' then 0.05 else hourly_precip end) as decimal(5,3)) as hourly_precip 
from default.weather
) x

In [ ]:
%%pyspark
weatherDF = sqlContext.sql("""
select distinct * from
(
select distinct 
month,
day,
latitude,
longitude,
ceil(time / 100) as hour,
cast((case wind_speed when null then 0.0 when 'M' then 0.0 else wind_speed end) as decimal(5,3)) as wind_speed,
cast((case sea_level_pressure when null then 0.0 when 'M' then 29.92 else sea_level_pressure end) as decimal(5,3)) as sea_level_pressure,
cast((case hourly_precip when '' then 0.0 when 'T' then 0.05 else hourly_precip end) as decimal(5,3)) as hourly_precip 
from default.weather
) x
""").cache()

In [ ]:
%%pyspark
display(weatherDF.head(5))

In [ ]:
weatherDF

In [ ]:
weatherDF.coalesce(1).write.mode('overwrite').format("csv").option("header", "true").save('abfss://sampledataset@REPLACETHIS.dfs.core.windows.net/flight/weather-ref/REPLACETHISWITHALIAS/AirportWeatherReferenceData.csv')